# Using Web Search Tool for Real-Time Information (LlamaStack 0.3.0)

## Web Search Integration

### Why Web Search?
- **Real-Time Data**: Access current information not in documents
- **Broader Knowledge**: Search the entire web, not just ingested documents
- **Dynamic Updates**: Information updates automatically

### Implementation
We'll use LlamaStack's web search tool which:
- Searches the web using Tavily search API
- Returns relevant web pages and snippets
- Integrates seamlessly with agents

### Tool Format in 0.3.0
In LlamaStack 0.3.0, the Agent uses the OpenAI-compatible responses API:
```python
tools=[{"type": "web_search"}]
```

### Setup Requirements
- **API Key**: Tavily search API key (set as environment variable or fallback value)
- **Provider Data**: Pass API key to LlamaStackClient for web search access

In [8]:
# Install notebook dependencies (LlamaStack 0.3.0)
# Will take a while to download and install numerous dependencies. 
# Wait until it finishes before proceeding
%pip install llama_stack_client==0.3.0


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
# Python stdlib imports
import os
import json
from datetime import date, datetime, timedelta
import re
import logging

# Suppress verbose and noisy HTTP logs
logging.getLogger("httpx").setLevel(logging.WARNING)

# Llamastack imports (0.3.0)
from llama_stack_client import LlamaStackClient
# Agent from lib.agents.agent uses responses API with OpenAI-compatible tool format
from llama_stack_client.lib.agents.agent import Agent

import rich

In [ ]:
# LlamaStack service URL (in-cluster)
LLAMASTACK_URL = "http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321"

# For access from Notebooks external to the cluster, use the route URL instead
# LLAMASTACK_URL = "https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/"

# Get Tavily search API key from environment variable
# Tavily is a search API that provides web search capabilities
tavily_search_api_key = os.getenv('TAVILY_SEARCH_API_KEY', 'tvly-xxxxxx')

# Configure provider data for web search
# If API key is available, pass it to enable web search functionality
if tavily_search_api_key is None:
    provider_data = None  # Web search will not be available
else:
    # provider_data: Configuration for external service providers
    # tavily_search_api_key: API key for Tavily search service
    provider_data = {"tavily_search_api_key": tavily_search_api_key}

# Initialize client with provider data for web search (0.3.0 API)
# provider_data enables the client to use external services like Tavily
client = LlamaStackClient(
    base_url=LLAMASTACK_URL,
    provider_data=provider_data,  # Enables web search if API key is provided
    timeout=300.0  # Extended timeout for web search operations
)

# Get available models
models = client.models.list()
model_id = next(m.identifier for m in models if m.model_type == "llm")

rich.print(f"Using model: {model_id}")

Using model: vllm-inference/granite-3-3-8b-instruct

In [11]:
# Verify that tavily search is available as a tool in Llamastack
# In 0.3.0, you can still use toolgroups.list() to check available tools
# Look for 'builtin::websearch' provider which enables the web_search tool type
toolgroups = client.toolgroups.list()
for tg in toolgroups:
    print(f"  - {tg.identifier} (provider: {tg.provider_id})")

  - builtin::rag (provider: rag-runtime)
  - builtin::websearch (provider: tavily-search)


In [13]:
# Create Agent with web_search tool (0.3.0 API)
# In 0.3.0, the Agent class uses the responses API which expects OpenAI-compatible tools
agent = Agent(
    client,  # LlamaStack client
    model=model_id,
    # Instructions should mention the tool so the model knows to use it
    instructions="You are a helpful assistant that uses web search to find real-time information. Always search the web when asked about current prices, news, exchange rates, or any real-time data.",
    # Use web_search tool in OpenAI-compatible format
    tools=[
        {"type": "web_search"}
    ],
)

# Sample queries - uncomment one to test
#query = "What is the latest news from the National Stock Exchange (NSE) of India?"
query = "What is the latest exchange rate between USD and INR?"

# Create a session and run the query
session_id = agent.create_session("tavily-session")
rich.print(f"[cyan]Processing user query:[/cyan] {query}")

# Create a turn (non-streaming for simpler response handling)
response = agent.create_turn(
    messages=[
        {"role": "user", "content": query}
    ],
    session_id=session_id,
    stream=False  # Get complete response
)

# Extract the response text (0.3.0 uses output_text property)
rich.print(f"\n[green]Response:[/green]")
rich.print(response.output_text)

Processing user query: What is the latest exchange rate between USD and INR?

Response:

As of the latest data, the exchange rate between USD and INR is:

1 USD is approximately 89.786 INR according to [Investing.com](https://www.investing.com/currencies/usd-inr),
1 USD is approximately 89.85 INR for central bank trading, as stated by [XE Currency 
Converter](https://www.xe.com/en-us/currencyconverter/convert/?Amount=1&From=USD&To=INR),
and 1 USD is exchanging at 84.27 INR as mentioned on 
[Compareremit](https://www.compareremit.com/todays-best-dollar-to-rupee-exchange-rate/), though please note that 
exchange rates can fluctuate and it's always a good idea to check a reliable financial news service for the exact 
current rate. 

Please verify from a financial news source for the most current rate as these values are subject to change.

## Additional Query Example

Try another query using the same agent session:


In [14]:
# Additional query using the same agent
# The agent maintains conversation context within the same session

query2 = "What are the latest tech news headlines today?"
rich.print(f"[cyan]Processing query:[/cyan] {query2}")

# Use the same session_id to maintain conversation context
response2 = agent.create_turn(
    messages=[
        {"role": "user", "content": query2}
    ],
    session_id=session_id,  # Same session as before
    stream=False
)

# Display the response
rich.print(f"\n[green]Response:[/green]")
rich.print(response2.output_text)


Processing query: What are the latest tech news headlines today?

Response:

Here are some latest tech news headlines today:

1. According to [Reuters](https://www.reuters.com/technology/), "GameStop announces new stock option award for CEO 
Cohen and Qualcomm in talks with Samsung Electronics for contract manufacturing, South Korean newspaper says."

2. On [Wired](https://www.wired.com/), you can read about "CES: Jensen Huang Says Nvidia's New Vera Rubin Chips Are
in 'Full Production', Google Gemini Is Taking Control of Humanoid Robots on Auto Factory Floors, and Lego's". 

3. [CNBC Technology](https://www.cnbc.com/technology/) covers "Fed's Kashkari says AI is causing a hiring slowdown 
in big companies" and the latest on "Venezuela, Tesla's second-place".

4. [GeekWire](https://www.geekwire.com/) reports "Tech Moves: Amazon AI leader joins Google Cloud; Meta taps new 
chief legal officer from Microsoft and Ring around the parking lot: Amazon's security company."

5. [CNN Business](https://www.cnn.com/business/tech) provides a wrap-up of "Gadget and smartphone trends and 
insights from tech industry leaders."

Please visit the above links for more detailed articles.

## Alternative: Using Responses API (Experimental)

The new Responses API in LlamaStack 0.3.0 provides an OpenAI-compatible interface. This approach uses `{"type": "web_search"}` tool format.

**Note**: This may or may not work depending on your LlamaStack server configuration. The `builtin::websearch` approach above is the recommended method.


In [15]:
# Using Responses API for web search (simplest approach)
# This is a single API call that handles everything

simple_query = "What is the current price of Bitcoin?"

# Use the responses.create() API with web_search tool
response = client.responses.create(
    model=model_id,
    input=simple_query,
    tools=[
        {
            "type": "web_search"  # Built-in web search tool
        }
    ],
)

# Get the response text
rich.print(f"[cyan]Query:[/cyan] {simple_query}")
rich.print(f"\n[green]Response:[/green]")
rich.print(response.output_text)


Query: What is the current price of Bitcoin?

Response:

According to various sources, the current price of Bitcoin (BTC) is around 91,820 USD. However, prices fluctuate 
quickly, and as of January 7, 2026, 7:33 am EST, it has fallen by 0.38% in the past 24 hours. For the most accurate
and up-to-the-minute details, please refer to a live cryptocurrency exchange platform.